In [111]:
from bs4 import BeautifulSoup as bs
import requests as rs
import re
import csv
import time
import json

In [22]:
def get_articles(url):
    resp = rs.get(url=url,cookies={'over18':'1'})
    soup = bs(resp.text,'lxml')
    prev_link = soup.find('div','btn-group-paging').find_all('a')[1]
    prev_link = prev_link['href'] if 'href' in prev_link.attrs else None
    
    positive = []
    negative = []
    for div in soup.find_all('div','r-ent'):
        href = div.find('div','title').a['href']
        title = div.find('div','title').text.strip()
        if re.match('\[.*\]',title):
            tag = re.match('\[.*\]',title).group(0)
            if '好' in tag:
                positive.append([title,href])
            if '爛' in tag or '負' in tag:
                negative.append([title,href])
    return prev_link,positive,negative

In [23]:
PTT_URL = 'https://www.ptt.cc'
start_url = PTT_URL + '/bbs/movie/search?q=黑豹'
prev_link, pos,neg = get_articles(start_url)
positive_posts.append(pos)
negative_posts += neg

In [24]:
while prev_link:
    url = PTT_URL + prev_link
    prev_link, pos, neg = get_articles(url)
    positive_posts.append(pos)
    negative_posts += neg
print(len(positive_posts),positive_posts[:3])
print(len(negative_posts),negative_posts[:3])
    

60 [[['[好雷] 《暴走曼哈頓》黑豹化身正義猛警抽絲剝繭', '/bbs/movie/M.1575041152.A.537.html']], [['[好雷]  黑豹 --其實蠻好看的', '/bbs/movie/M.1552276420.A.F6F.html']], []]
56 [['[負雷] 黑豹 你就是個輸不起的屁孩 （有雷）', '/bbs/movie/M.1585842616.A.70A.html'], ['[負雷]黑豹-失衡的烏托邦', '/bbs/movie/M.1529245622.A.AF5.html'], ['[負雷] 四不像的黑豹', '/bbs/movie/M.1527918611.A.56C.html']]


In [36]:
with open('mov_pos.csv','w',encoding='utf-8',newline='')as f:
    writer = csv.writer(f)
    writer.writerow(['title','href'])
    for pos in positive_posts:
#         print(pos)
        writer.writerows(pos)

In [32]:
with open('mov_neg.csv','w',encoding='utf-8',newline='')as f:
    writer = csv.writer(f)
    writer.writerow(('title','href'))
    for neg in negative_posts:
        writer.writerow((neg))

In [102]:
def sanitize (txt):
    expr = re.compile('[^\u4e00-\u9fa5。;；,，:：“”"" ()（）、?？「」【】\s\w:/\-.]')
    txt = re.sub(expr,'',txt)
#     txt = re.sub(r'[。;；,，:：]','',txt)
    txt = re.sub('[。;；,，:：“”"" ()（）、?？「」【】/\-.]','',txt)
    txt = re.sub('(\s)+','',txt)
    txt = txt.replace('--','')
    txt = txt.lower()
    return txt

In [103]:
def get_post(url):
    resp = rs.get(url=url,cookies={'over18':'1'})
    soup = bs(resp.text,'lxml')
    main_content = soup.find('div', id='main-content')
    for meta in main_content.find_all('div','article-metaline'):
        meta.extract()
    for meta in main_content.find_all('div','article-metaline-right'):
        meta.extract()
    for meta in main_content.find_all('div','push'):
        meta.extract()
    parsed = []
    for txt in main_content.stripped_strings:
        if txt[0] == '※' or txt[:2] == '--' or url in txt:
            continue
        txt = sanitize(txt)
        if txt:
            parsed.append(txt)
    return ''.join(parsed)


In [104]:
def get_article_body(csv_file):
    id_to_body = {}
    with open(csv_file, 'r', encoding='utf-8')as f:
        reader = csv.DictReader(f)
        for row in reader:
            print('處理',row['title'],row['href'])
            title = ''.join(row['title'].split(']')[1:])
            title = sanitize(title)
#             print(title)
            body = get_post('https://www.ptt.cc'+row['href'])
#             print(body)
            id_to_body[row['href']] = title+''+body
            time.sleep(1)
    return id_to_body

In [105]:
d1 = get_article_body('mov_pos.csv')
# print(title)
# print(body)


處理 [好雷] 《暴走曼哈頓》黑豹化身正義猛警抽絲剝繭 /bbs/movie/M.1575041152.A.537.html
暴走曼哈頓黑豹化身正義猛警抽絲剝繭
雷文防雷資訊頁雷文主文分隔線魯魯說電影_電影簡介_暴走曼哈頓由復仇者聯盟4終局之戰的羅素兄弟製作並由漫威英雄電影黑豹一砲而紅男星查德維克博斯曼chadwickboseman主演黑豹脫下英雄戰袍化身成機智警探準備開啟一場槍林彈雨的火爆對決暴走曼哈頓描述警探安德烈查德威克鮑斯曼飾的父親從警時因公慘遭殺害因此他誓言將所有殺警凶手繩之以法為了偵破一樁8名警察慘遭奪命的駭人搶劫案他必須在時限內抓到凶手於是不擇手段展開緝凶行動大規模封鎖曼哈頓全境21座橋並關閉所有交通系統但就在追尋正義的同時卻也揭開意想不到的黑幕_強強聯手_羅素兄弟加上黑豹的組合相信已經拉高不少期待值雖然片名有那麼一點點不吸引人但當正片一開始刺激緊張的槍擊對決加上撲朔迷離的懸疑劇情看得是熱血沸騰毫無冷場帶有社會議題性的題材更為本片增添不少可看性除了精彩的槍戰鏡頭抽絲剝繭般的劇情設計更是讓人一秒都捨不得離開大螢幕就怕錯過了甚麼重要的線索前面的鋪陳就白看了_正與邪_從最初的展開大略的敘述了查德維克博斯曼飾演的安德烈日後性格嫉惡如仇的原因來自於父親的逝世爾後又面臨執法過當的爭議卻始終堅持自己是正義的執行者一開始導演在安德烈的人設上就悄悄的為觀眾植入了部分的偏執_霸氣外漏_後來藉由一樁毒販搶劫衍生成的弒警案慢慢帶到安德烈雷厲風行的行事風格並且果斷而明智的執行著後續的案件追蹤其中最讓人感受到霸氣的命令非封鎖曼哈頓所屬在即將迎來凌晨的五六個小時內全城封鎖揚言非讓罪犯無處可逃最終乖乖落網那個氣魄真的是帥慘了_黑白難辨_此外由曾經執導復仇者的羅素兄弟監製的場景相信沒有讓大家失望無論是飛車槍戰還是地鐵追逐戰或是隔著監視孔爆眼的射擊華麗的動作場面都讓人彷彿身歷其境般一整個緊張到不行覺得胃都要收縮過頭了lol然而隨著槍聲案外案的漏洞也一個個地蹦了出來事情進行得太順利了彷彿有人刻意引導的犯案終於通通在一個隨身碟裡露了餡收賄的警員大宗的毒品運輸才是搶劫犯無意中拉扯出的巨大謎團_反思空間_整部片很有意思的是在你自以為的正義之中又有多少看不見的黑暗誓死捍衛的堅持之下又能否接納那些似是而非的犯案理由而當別人利用我的正義之時我又是否會為此感到憤怒猶如先前我們與惡的距離一樣每個立場都有自

處理 [微好雷]《雷神索爾3諸神黃昏》＆《黑豹》 /bbs/movie/M.1536217925.A.B6E.html
雷神索爾3諸神黃昏黑豹
雷神索爾3諸神黃昏黑豹雖然都已經不是院線片了但還是防雷一下開學前暑假的最後一個禮拜我終於補齊了在復仇者聯盟3無限之戰以下簡稱復3前我所遺漏的兩部漫威系列電影雷神索爾3諸神黃昏以下簡稱雷神3與黑豹雷神3的情節相當影響了復3不但復3的開場就緊接著雷神3的結尾當初沒看雷神3的我完全訝異於索爾的短髮鎚子不見以及他變成了一個獨眼龍雷神3的劇情結構相當簡單一開場索爾就去挑戰火焰惡魔蘇爾特爾surtur並將之打敗在和洛基一起找到失蹤的老爸奧丁時本集真正的魔王被封印多年的姐姐海拉hela出現索爾的鎚子成為姐姐的強度測試被捏爆在兄弟兩人要透過彩虹橋逃走的過程中被姐姐一腳踹到蟲洞另一頭的垃圾星球薩卡星索爾並被抓起來賣給星球統治者宗師grandmaster並變成他旗下鬥技場的一員但提早到的洛基卻和宗師成為朋友索爾在這裡發現浩克也是鬥技場成員甚至是衛冕冠軍總之最後索爾偕同浩克與當初抓他拿去賣的女武神查了之後才知道原來這是fgo中的布姐啊回到阿斯嘉對抗海拉洛基也帶回鬥技場的奴隸趕來助陣最後洛基發揮雷神之力並重新復活蘇爾特爾摧毀阿斯嘉破壞海拉的力量來源也就是引發所謂的諸神黃昏阿斯嘉的滅亡打敗海拉並帶著阿斯嘉的人民逃往地球但電影的第一個片尾彩蛋就是復3的大魔王薩諾斯的太空船出現在載著阿斯嘉人民的飛船旁之所以說雷神3的劇情結構簡單是因為這部電影就是遇見大魔王老姐被打敗丟入宇宙式的羅馬競技場再逃出競技場後二次面對魔王透過力量覺醒加上復活一開始打敗的對手以惡制惡獲得勝利然而儘管劇情結構簡單但仍讓作為觀眾的我覺得相當好看這或許是照著一種基本公式規規矩矩的走就可以提供給觀眾的穩定的娛樂相對於雷神3黑豹blackpanther則有比較多的象徵意義在美國隊長3英雄內戰首次出現黑豹這個角色以及他的神秘國度瓦干達wakanda而黑豹這部電影可說是把瓦干達這個神秘國度帶入漫威宇宙或說是漫威現實世界的作品為什麼我說是帶入漫威的現實世界呢理由是瓦干達作為一個非洲的高科技國家這個設定是很神話式且違逆現實的也就是我們對於非洲又落後又窮的認知在我看來瓦干達的高科技可以說是所謂西方世界亞特蘭提斯神話超古代高科技文明的一個變形一個非洲版的亞特蘭提斯要讓這個神話國度在漫威宇宙中合理化就必與

處理 [ 好雷] 唯一缺憾的黑豹 /bbs/movie/M.1519908058.A.B99.html
唯一缺憾的黑豹
如果春節還沒看很推喔防雷黑豹唯一讓我開始抽離的地方是帝查卡第二次跟帝查拉對話的時候帝查拉說出no的剎那整個超抽離而且沒什麼背景音樂臉部的特寫背景的渲染感突然出現抽離到我開始注意電影院的裝潢配置抱怨一下我看的場次很多字都有殘影不知道是不是影片本身的問題還是影廳的播放品質
處理 [好雷] 黑豹 感想 微負評 /bbs/movie/M.1519841715.A.3BF.html
黑豹感想微負評
先說好的部分一個全新的世界觀一個全新的社會融合了傳統習俗與高科技從驚艷的服裝設計到3d立體動畫一個能隨時穿戴在身上方便酷炫的黑豹裝甲還有飛車追逐戰高科技科幻兼容原始美學的城市光是這些美術設計就很值得一看接下來是負雷雖說是負雷可是正確的來說是讓我看了覺得有衝突的地方而一個電影就是有著某種程度的不合理與衝突才是好看的地方以下是我的感想即便都是臭幹也是值得一看的地方比方說整個劇中的烏干達就是一個科技發達可是政治體制脆弱完全人治的國家後來他表兄金豹跑回來討王位我感覺合情合理感覺都想跳起來大喊金豹加油相對之下黑豹的家人私通外國間諜祖父輩無能皇室成員獨佔所有知識資源與能力整個國家祇有黑豹能當王不是他很行是因為你們只有教他一個人後來有一段黑豹家人逃亡後來逃到另個部族我看到就笑尿了幹這跟你們首都差得有夠多貧富懸殊不難理解而且他們國王三十年來從來沒鳥過這部落一來就要你去送死不只要你死還要出錢出力當工具人贏了沒你的份輸了都算你幹媽的臭不要臉話都給你講就好所以首領也很聰明幫你們救回黑豹讓你們自己去鬥要軍隊做夢還找外人去殺自己人咧你以為自己是路易十六哦老子抓到你就獻給金豹討賞了啦北七最後的大戰更是讓我看了悲痛萬分就你們為了爭權讓國家發生內戰誰才是壞人我看主角方跟壞人方都很爛這種內部糾紛會多大的危害百姓知道嗎你們心中有百姓嗎這麼多的問題就是你們把一個孤兒仍在國外造成的可恥不可恥最後才在那邊演一個開放國家的明君說真的要不是劇組給金豹一個對外發動戰爭的藉口讓他當壞人要不然金豹比較適合當主角金豹萬歲去你黑豹sentfromjpttonmyxiaomimimax2
處理 [核心好雷] 黑豹-科技與部落的反差萌 人權與人道議題 /bbs/movie/M.1519812809.A.DC3.html
黑

處理 [ 好 雷] 黑豹 /bbs/movie/M.1519572758.A.D3C.html
黑豹
為初二跟老妹阿姨表弟妹們六人去看不過因為一些事故拖到現在才推看完電影六人普遍反應都算okm社電影表弟妹跟阿姨之前只看過雷神3而我妹沒看過m社的任何一部電影甚至在看之前還把英雄片誤以為是浴血任務二同類型的電影因此之前非常排斥英雄片現在也開始對m社電影有興趣她也說等會考結束後再慢慢補m社的電影我妹從以前就滿喜歡克里斯伊凡因此我先從美隊推起加強審美而阿姨與表弟妹們對於雷神3跟黑豹這兩部風格迥異的電影風格都相當喜歡這樣的結果老實說我滿驚訝的以後推人可以用美隊系列或黑豹推了現在說我的感想老實說就故事性與節奏而言我認為這部電影只有中上但是算是我在m社看過最嚴謹的設定並順著設定好好講完一個故事沒太多bug首先先從我認為的缺點開始說起好了我認為黑豹的整體的缺點在於1失敗的武打戲原本算是略顯沉悶的文戲在失敗的武戲的襯托下導致後來金黑兩豹在最後的決鬥畫面看起來像這樣httpsimgurcomufz6qrl導致雙方的理念鬥爭看起來太智障而導致整個節奏被打亂2過於簡略的敘事發展講好聽的為沒什麼超展開而講難聽點為過於匆忙的敘事導致電影後期非常的趕同時也使得一些可以深論的議題沒有好好討論ex黑豹該如何定義金豹理念的價值如何看待類似金豹流落在異邦的瓦干達人等電影就快速的金豹奪權遭到反抗便當而導致金豹的奪位行動看起來很兒戲在來說說我喜歡這部電影的部分甚至願意給到中上的原因1演員這部有太多演員成功是演出令人印象深刻的地方無論是女將軍的威武公主的俏皮母后的貴氣等但是最為成功的還是飾演黑豹與金豹的兩位演員最為精彩黑豹常常被酸在女主群的發揮下失去原本在美隊3裡的王霸之氣但是我認為在金豹劫持無良商人克勞以前與部落族長的交談追捕克勞與羅斯的交涉都有顯現出王者的風範直到金豹的出現與祖利的對質對於父親統治的困惑不曉得如何面對這位多出來的堂兄弟直到金豹出現代節奏奪取王位因在短時間的迷惑而失去王位到事件落幕後排除心中疑慮在聯合國會議討論分享瓦干達科技從新恢復王者風範都有明顯的顯現出一位善良的學習人如何當一位成功國王的歷程而金豹的演員即便在這兒戲般的奪權行動但透過他的演技成功敘述出流落在異邦外的瓦干達人的苦痛與對世道的不滿在即便這麼兒戲安排依然不失於一個反派的格調該執行的步驟執行該殺的人就殺絲毫不猶豫成功顯現出一個反派

處理 [微好雷]《黑豹》　春秋五霸的基本套路 /bbs/movie/M.1518991149.A.BDD.html
黑豹春秋五霸的基本套路
歡迎支持部落格原文httpsgooglxk5tjg以下有雷黑豹標準的動作爽片動作場面流暢劇情轉折合理將焦點完全放在瓦干達跟黑豹的故事沒有多餘的支線可以視為一部獨立作品即使沒看過漫威其他作品也無礙全片表現中規中矩沒啥重大缺點雖然也沒有令人印象深刻的爆點但作為一部爽片而言它娛樂性相當夠分數絕對有及格可惜的是漫威優秀的作品不少珠玉在前少了爆點的黑豹便顯得不夠出眾本片的劇情條理分明沒有燒腦的邏輯思維或驚人的峰迴路轉用保守的套路講述黑豹帝查拉查德維克博斯曼飾與瓦干達的故事不過許多人認為劇情的鋪排上有迪士尼成名作品之一獅子王的影子其實追溯更遠這類的王子復仇記在中國春秋戰國時代屢見不鮮左傳裡便有諸多記載公子小白跟公子重耳都曾經逃亡而後返國奪回政權前者就是大名鼎鼎的齊桓公後者也是頗具盛名的晉文公兩者皆列名為春秋五霸黑豹劇情無創新所幸能緊扣主題無多餘拖戲的場景這點優於雷神索爾3有太多刻意扮蠢搞笑以及星際異攻隊2小格魯特過度賣萌適量的扮蠢跟賣萌雖能增加點娛樂性但過多則讓人顯得厭煩在片長方面灌水硬湊分鐘數黑豹比較明顯的缺點在於同樣都是黑豹動作流暢的武戲精采程度卻比不上美國隊長3裡的片段美國隊長3裡的黑豹設計的動作靈活俐落出招快準狠不論是公路那場跟美國隊長的追擊戰或是停機坪的大亂鬥武打招式充滿力與美令人激賞反觀本片的黑豹除了在韓國那場飛車追戲之外其他的武打場面皆無驚艷之感甚至招式上面有些捉襟見肘像是帝查拉第一場繼承王位的對決有一招後空翻踢腿用拉近鏡頭拍攝特寫但用剪接方式呈現出空翻跟踢腿兩幕非一鏡到底少了一氣呵成的感覺動作也不夠清楚最後決戰的武打也無耳目一新的招數難留下深刻的印象儘管劇本方面無創新武戲差強人意黑豹仍不算是一部糟糕的作品因為本片劇情結構完整起承轉合面面俱到對於人物性格的塑造十分成功更難能可貴的是除了男主角與大反派其餘配角也都各有特色女將軍黑豹親妹黑豹前女友和邊境守將等每一位皆是性格鮮明的角色足以在觀眾心中留下一席之地另外漫威的反派設定的老毛病依舊存在鋼鐵人和蟻人的反派皆穿上與主角類似的裝備黑豹亦承襲此點太過公式化還好反派角色性格突出身分背景也叫人同情與齊莫男爵禿鷹有相似之處並非全然的窮凶極惡之徒能引起觀眾對於善惡的反思最後用一首十四行詩點評

...

處理 [微好雷]《黑豹》　春秋五霸的基本套路 /bbs/movie/M.1518991149.A.BDD.html
黑豹春秋五霸的基本套路
歡迎支持部落格原文httpsgooglxk5tjg以下有雷黑豹標準的動作爽片動作場面流暢劇情轉折合理將焦點完全放在瓦干達跟黑豹的故事沒有多餘的支線可以視為一部獨立作品即使沒看過漫威其他作品也無礙全片表現中規中矩沒啥重大缺點雖然也沒有令人印象深刻的爆點但作為一部爽片而言它娛樂性相當夠分數絕對有及格可惜的是漫威優秀的作品不少珠玉在前少了爆點的黑豹便顯得不夠出眾本片的劇情條理分明沒有燒腦的邏輯思維或驚人的峰迴路轉用保守的套路講述黑豹帝查拉查德維克博斯曼飾與瓦干達的故事不過許多人認為劇情的鋪排上有迪士尼成名作品之一獅子王的影子其實追溯更遠這類的王子復仇記在中國春秋戰國時代屢見不鮮左傳裡便有諸多記載公子小白跟公子重耳都曾經逃亡而後返國奪回政權前者就是大名鼎鼎的齊桓公後者也是頗具盛名的晉文公兩者皆列名為春秋五霸黑豹劇情無創新所幸能緊扣主題無多餘拖戲的場景這點優於雷神索爾3有太多刻意扮蠢搞笑以及星際異攻隊2小格魯特過度賣萌適量的扮蠢跟賣萌雖能增加點娛樂性但過多則讓人顯得厭煩在片長方面灌水硬湊分鐘數黑豹比較明顯的缺點在於同樣都是黑豹動作流暢的武戲精采程度卻比不上美國隊長3裡的片段美國隊長3裡的黑豹設計的動作靈活俐落出招快準狠不論是公路那場跟美國隊長的追擊戰或是停機坪的大亂鬥武打招式充滿力與美令人激賞反觀本片的黑豹除了在韓國那場飛車追戲之外其他的武打場面皆無驚艷之感甚至招式上面有些捉襟見肘像是帝查拉第一場繼承王位的對決有一招後空翻踢腿用拉近鏡頭拍攝特寫但用剪接方式呈現出空翻跟踢腿兩幕非一鏡到底少了一氣呵成的感覺動作也不夠清楚最後決戰的武打也無耳目一新的招數難留下深刻的印象儘管劇本方面無創新武戲差強人意黑豹仍不算是一部糟糕的作品因為本片劇情結構完整起承轉合面面俱到對於人物性格的塑造十分成功更難能可貴的是除了男主角與大反派其餘配角也都各有特色女將軍黑豹親妹黑豹前女友和邊境守將等每一位皆是性格鮮明的角色足以在觀眾心中留下一席之地另外漫威的反派設定的老毛病依舊存在鋼鐵人和蟻人的反派皆穿上與主角類似的裝備黑豹亦承襲此點太過公式化還好反派角色性格突出身分背景也叫人同情與齊莫男爵禿鷹有相似之處並非全然的窮凶極惡之徒能引起觀眾對於善惡的反思最後用一首十四行詩點評

處理 [微好雷] 黑豹的電影配樂 /bbs/movie/M.1518848376.A.BBB.html
黑豹的電影配樂
剛剛看完黑豹的確在編劇上我認為他是有許多不足的包括他們戰鬥的理由跟一些不合理的細節前面有許多討論這裡不多詳述但我仍然會給這部片微好雷其中的原因不只是因為他的畫面感跟美術做的非常好感覺的出來很用心也的確非常精緻另一個讓我喜歡這部片的點的是整部電影的配樂跟非洲遼闊的原野風景搭配的相得益彰民族音樂融入trap的鼓點重新remix也讓我有很驚艷的感覺各個段落中配樂下的也是頗為準確我覺得這是另一個讓這部電影不至於被歸類到難看的原因甚至我個人的觀點還傾向是還不錯還是會推薦大家進電影院看啦畢竟用那個音響聽配樂真的太爽ps麥克b喬丹真的有夠帥在看電影的時候一直想到金牌拳手xd不知道484他混得欸
處理 [好無雷] 黑豹 /bbs/movie/M.1518802967.A.D32.html
黑豹
剛看完黑豹看到ptt這麼多給黑豹負累的就知道ptt觀影結構有多低落以看過的漫威電影來說s美隊2s美隊3雷神3gog1a鋼鐵人1gog2復仇者1b級以下不列這部黑豹至少有a的水準完整的劇情好的起承轉合不錯的打鬥美術及特效畫面演員的平均稱職表現出色的反派沒有讓人無言的漫威式刻意搞笑結果居然有ptt影評說不如蟻人跟奇異博士天啊拜託觀影能力低落別隨便亂發影評好嗎半桶水響叮噹不說話還沒人笑你呢gog1當然大於gog2gog1是先行開拓者請問第二集站在第一集的肩膀突破了什麼就是一個挺有魅力的反派跟勇渡便當而已黑豹可以用出色形容中規中矩是指美隊1蟻人這種噗無敵浩克的爛番茄跟imdb先看看好不好普通好猜的漫威劇情多的是鋼鐵人1從光頭露臉就知道他是反派復仇者1從還沒進電影院就知道集合打洛基然後洛基被痛扁真正有創意劇情的只有美隊2美隊3gog1蛤我們看的是同一部黑豹還是又一個人云亦云的ptt影評家你沒有鑑賞能力乖乖睡覺是應該不我是指你這種ptt影評家服裝脈衝整部電影不超過10次頻繁笑死整部電影能讓你批評的只有cg我看你還是先把嘴巴閉上吧又是一個人云亦云的ptt影評家呢什麼觀影水準要不要查查黑豹爛蕃茄影評評分因為大多數ptt觀影者能力低落阿這是明顯客觀現實不用玻璃心因為這些是爛意見阿爛意見就乖乖閉嘴很難嗎你別說笑話啦ptt觀影能力在以前是還不錯現在混一些不三不四沒有鑑賞能力又愛昭告天下自己觀影能

處理 [普好雷] 覺得黑豹的編劇陷入一種兩難的情況 /bbs/movie/M.1518632731.A.55E.html
覺得黑豹的編劇陷入一種兩難的情況
看完覺得這點很明顯反派艾瑞克齊爾蒙格的故事比帝查拉有趣多了雖然也是復仇舊梗但就是比較立體抑或是帝查拉的故事實在太一般跟艾瑞克齊爾蒙格比起來就讓人索然無味劇情上很多張力都是靠金錢豹耍狠帶起來我覺得這時候黑豹的編劇陷入了一種兩難的境界第一種劇情如果幾乎聚焦在比較有戲的金錢豹劇情會比較好但是會讓黑豹被邊緣化更不用說當漫威看到成品時應該會驚呆了第二種不能讓金錢豹整個蓋過主角黑豹只好用一些很彆扭的方式去平衡雙方劇情然而這樣做會讓劇情上有些不順就沒那麼優秀又因為不想做的太明顯所以就分散在各種段落一點一點的去修正我想許多人所說的劇情瑕疵就是因為這樣而產生看這結果想當然爾編劇選擇了後者畢竟錢是漫威出的你總不能演到最後結果黑豹的片尾變成金錢豹將會在無限之戰回歸吧囧我覺得黑豹這部片還是能看ｏｋ的說他是雷神索爾１太過了黑豹比雷神索爾有內容多了只是他那個內容因為妥協寫的不夠優秀整體而言就是不空洞但是沒寫到深處觸動到觀眾的心這樣艾瑞克齊爾蒙格的終末帶給觀眾應該是更悲壯的心情因為前面各種不能搶過黑豹風采搞的最後那幕不上不下看完後其他人吐槽黑豹的史上最佳評語我真的沒意見因為以起源電影來說蟻人黑豹雷神個人觀感大概是這個差距
處理 [好笑雷] 黑豹 /bbs/movie/M.1518627430.A.ADF.html
黑豹
山頂哪個種族很好笑美國人在他們部落要插話時山頂族就一直嗚嗚嗚叫有人覺得這段好笑嗎我在想說如果彩蛋1媒體問說你們農業國家能為世界改變甚麼然後瓦甘達的人同時嗚嗚嗚這樣會不會更好至少會讓大家笑得很開心
處理 [好雷] 黑豹 耳目一新的英雄電影 /bbs/movie/M.1518617014.A.223.html
黑豹耳目一新的英雄電影
防雷這篇是重po的因為昨天回覆貼文後好像編輯錯了結果誤砍推文跟一堆內文重po應該沒違反版規吧板上負雷文很多缺點大家都列出許多了但當下看完這部感受蠻好的給個好雷這真的是一部美國黑人電影不只因為演出者及製作團隊多為黑人或背景設定在非洲而是因為整部電影進行都圍繞著反派並緊扣著美籍非裔的困境該命題這部片中瓦干達就像是非裔的桃花源在瓦干達沒有種族間的解構性壓迫他們是一個強大富饒的國家擁有全世界望塵莫及的科技艾瑞

處理 [好雷] 不太懂有關黑豹反派的幾件事情 /bbs/movie/M.1518512928.A.747.html
不太懂有關黑豹反派的幾件事情
其實我看片名字都記不太住不過反派也就兩個吧一個是盜拿汎合金的斷臂白人商人一個是要會去奪王位的堂弟堂哥先講白人既然只是要交易貴重物品怎麼不讓小弟出馬就好他好像有講到可以引出黑豹他們不過他們並沒有有所作為啊再來就是賣給fbi是怎樣想被抓嗎第二個是有關黑人cousin他老爸算是叛國這個罪刑可以誅九族的為什麼黑豹跟祭司都不說還要國王應該要有國王的樣子啊人家說要挑戰你就說好喔題外話黑人cousin要黑豹問他是誰時他拒絕結果後面一個老頭居然問了wtf真的國王當假的歹勢沒注意到
處理 [好雷] 小細節怪怪的黑豹 /bbs/movie/M.1518511893.A.636.html
小細節怪怪的黑豹
防雷總體來說黑豹主軸明確特效亮眼動作戲爽快刺激配樂有時有些突兀但振奮人心衝著高科技非洲部落風都值得看但是劇情細節卻有些黑人問號例如說老爸殺人說要保密卻不處理屍體和文件豹哥聽到老爸殺叔叔馬上大崩潰電影起碼演一下你們感情很好不行嗎探員就算替你擋槍也要冒著國安危機救他已經結束加冕還可以因為有人本來有權就搶王位而且你好像忘了他可是把你們的仇敵直接救走就算帶屍體來又怎樣負責儀式的大祭司說殺就殺國王上任可以隨便殺舊王家屬和燒國寶心草你們大臣怎麼當的雪山部族忘記名字以為是權力慾者結果居然送佛送上西反到倒另一個部族表面笑笑卻翻臉如翻書親衛隊也是嘴巴好聽說個話就長茅相向以上幾點給我的感覺就是大綱沒問題每個段落接起來卻很怪異這樣但還是非常值得看的一部電影尤其是復仇者三有瓦甘達大軍當前傳看是必要的還有妹妹公主超級可愛
處理 [好雷] 黑豹 /bbs/movie/M.1518507928.A.ABD.html
黑豹
今天去看了看完之後覺得還好大概是普偏好雷吧劇情敘述瓦干達王子帝查拉繼承王位之後突然有另一位王族血統的人來奪權導致瓦干達發生內戰最後帝查拉大殺四方成功奪回王位的故事跟另外一位復仇者聯盟成員的經歷有點像啊xd對了我一覺睡起來很多角色的名字都忘光了請見諒主角帝查拉在美隊3裡一直氣噗噗地追著巴奇跑遍半個地球總是一臉不爽但在這部個人電影裡可以看到他幽默和溫柔的一面他一開始就對堂弟的遭遇感到憐憫而且在經歷過齊默事件之後深知仇恨造成的災難有多強大所以對待堂弟感覺就是很

處理 [好雷] 美國隊長3  黑豹家世 /bbs/movie/M.1461748087.A.515.html
美國隊長3黑豹家世
美國隊長看來是漫威裡面最成功的系列對人物的課劃劇情的佈局都做得非常好蠻喜歡這次的新角色黑豹沒想到最後把美國隊長撿走的是黑豹小弟平時除了看看漫威電影外沒在接觸漫威漫畫想問一下黑豹是什麼家世我知道他是國王啦只是沒想到結局那邊怎麼搞到像東尼那種財團一樣漫威下一部不知道是蜘蛛人還是黑豹還有雖然以前看福爾摩斯還蠻喜歡費里曼的可是現在看到他在每部電影都在演同一個人看到有點反感


In [106]:
print(d1)

{'/bbs/movie/M.1575041152.A.537.html': '暴走曼哈頓黑豹化身正義猛警抽絲剝繭雷文防雷資訊頁雷文主文分隔線魯魯說電影_電影簡介_暴走曼哈頓由復仇者聯盟4終局之戰的羅素兄弟製作並由漫威英雄電影黑豹一砲而紅男星查德維克博斯曼chadwickboseman主演黑豹脫下英雄戰袍化身成機智警探準備開啟一場槍林彈雨的火爆對決暴走曼哈頓描述警探安德烈查德威克鮑斯曼飾的父親從警時因公慘遭殺害因此他誓言將所有殺警凶手繩之以法為了偵破一樁8名警察慘遭奪命的駭人搶劫案他必須在時限內抓到凶手於是不擇手段展開緝凶行動大規模封鎖曼哈頓全境21座橋並關閉所有交通系統但就在追尋正義的同時卻也揭開意想不到的黑幕_強強聯手_羅素兄弟加上黑豹的組合相信已經拉高不少期待值雖然片名有那麼一點點不吸引人但當正片一開始刺激緊張的槍擊對決加上撲朔迷離的懸疑劇情看得是熱血沸騰毫無冷場帶有社會議題性的題材更為本片增添不少可看性除了精彩的槍戰鏡頭抽絲剝繭般的劇情設計更是讓人一秒都捨不得離開大螢幕就怕錯過了甚麼重要的線索前面的鋪陳就白看了_正與邪_從最初的展開大略的敘述了查德維克博斯曼飾演的安德烈日後性格嫉惡如仇的原因來自於父親的逝世爾後又面臨執法過當的爭議卻始終堅持自己是正義的執行者一開始導演在安德烈的人設上就悄悄的為觀眾植入了部分的偏執_霸氣外漏_後來藉由一樁毒販搶劫衍生成的弒警案慢慢帶到安德烈雷厲風行的行事風格並且果斷而明智的執行著後續的案件追蹤其中最讓人感受到霸氣的命令非封鎖曼哈頓所屬在即將迎來凌晨的五六個小時內全城封鎖揚言非讓罪犯無處可逃最終乖乖落網那個氣魄真的是帥慘了_黑白難辨_此外由曾經執導復仇者的羅素兄弟監製的場景相信沒有讓大家失望無論是飛車槍戰還是地鐵追逐戰或是隔著監視孔爆眼的射擊華麗的動作場面都讓人彷彿身歷其境般一整個緊張到不行覺得胃都要收縮過頭了lol然而隨著槍聲案外案的漏洞也一個個地蹦了出來事情進行得太順利了彷彿有人刻意引導的犯案終於通通在一個隨身碟裡露了餡收賄的警員大宗的毒品運輸才是搶劫犯無意中拉扯出的巨大謎團_反思空間_整部片很有意思的是在你自以為的正義之中又有多少看不見的黑暗誓死捍衛的堅持之下又能否接納那些似是而非的犯案理由而當別人利用我的正義之時我又是否會為此感到憤怒猶如先前我們與惡的距離一樣每個立場都有自己的盲點也有自己的辯駁對我來說利用他人正義感執行

In [94]:
    expr = re.compile('[^\u4e00-\u9fa5。;；,，:：“”"" ()（）、?？「」【】\s\w:/\-.]')
    txt = re.sub(expr,'','黑豹:doyouknowdaway?')
    txt = re.sub(r'[。;；,，:：?？]','',txt)
    txt = re.sub('(\s)+','',txt)
#     txt = txt.replace('--','')
#     txt = txt.lower()

In [107]:
d2= get_article_body('mov_neg.csv')

處理 [負雷] 黑豹 你就是個輸不起的屁孩 （有雷） /bbs/movie/M.1585842616.A.70A.html
黑豹你就是個輸不起的屁孩有雷
雷文防雷資訊頁雷文主文分隔線一個歷史悠久的國家領導人選出來的辦法竟然是決鬥黑豹編劇是腦子進水這種誰打架厲害就能統治國家的邏輯還真夠非洲的編劇還把好好一個黑豹搞成輸不起的屁孩輸不起就算了背叛國王搞政變讓忠於國王的部族只因聽命合法國王就被屠殺這叫文明國家喔我看是未開化部落吧全片就是一流特效二流導演三流演員不入流編劇做成的爛片全部漫威系列最爛沒有之一原本是雷2
處理 [負雷]黑豹-失衡的烏托邦 /bbs/movie/M.1529245622.A.AF5.html
黑豹失衡的烏托邦
以下有黑豹和影集亞特蘭大雷請慎入以下是影集亞特蘭大雷當時看完黑豹馬上就能理解為數不少的負評和普評是從何而來看完的第一感想雖然不至於覺得爛但完全就是滿頭問號整個故事怎麼能經營成這樣我立刻就想到影集亞特蘭大第1季某集的劇情唐納葛洛弗飾演的主角他也是該劇製作人編劇近期有推出熱門單曲thisisamerica及飾演韓索羅的藍多和妻子在那一集為了事業前途的發展找到機會去參加某個有錢人的的派對那個有錢白人非常熱愛並崇拜黑人文化上從歷史層面的非洲部落藝術到當今流行嘻哈文化全部瞭若指掌政治面上極度崇拜金恩博士和馬爾坎x等人權運動者還會以他們為靈感作畫寫詩整個家擺放了大量的非洲藝術品和經典的歷史照片和嘻哈潮流商品有錢人在席間和主角提到他去了一趟非洲懺悔及朝聖的文化之旅非常有意義的一趟旅程也建議主角可以去一趟非洲了解祖先和文化真的會很感動很有價值到了該集的最後總之主角受不了這個派對上的各種事打算要離開他終於憋不住說出口全部都太胡扯了這間別墅的一切一切都太詭異了你跟我所說的都不是真正的生活啊你跟我說我應該去非洲文化之旅這不可能我他媽的快窮死了還非洲之旅然後他和妻子留下錯愕的主人逃跑了我知道迪士尼對文化的選用和對非洲未來主義的風格花了很多心思但我看完總是想到這個對黑人文化非常理解的有錢人亞特蘭大雷結束一開始看完的時候不是很懂為什麼導演掌握的那麼奇怪角色魅力的分配不均衡議題和故事也沒有緊密貼合要說的道理真的有順著劇情發展嗎主角的成長弧線也蠻詭異的怎麼會這樣這真的是年紀輕輕就能拍出奧斯卡的一天和金牌拳手的同一位導演後來想想他之前沒出錯的故事在黑豹裡也沒失手主要角色群中的唯

處理 [負雷]黑豹  最差漫威電影 /bbs/movie/M.1519931938.A.207.html
黑豹最差漫威電影
標題很聳動但這就是我的內心話實際觀影體驗就是看到一半覺得煩悶會想知道電影已經演多久了同行的朋友更是睡到不知神遊去哪裡打呼聲超清楚黑豹或許是黑人族群的美麗童話你可以盡情聯想黑人的脈絡可以盡情參考黑豹漫畫最深度最有錢最快破紀錄最一堆最但是單就電影本身黑豹的內容根本就乏善可陳卸下那些腦補的美麗聯想黑豹只是套用簡單的電影公式的便宜行事之作根本經不起考驗黑豹對漫威系列來講它製作上的粗糙絕對是有目共睹看到這邊相信很多人應該準備要按2了但請讓我娓娓道來為什麼要這樣講我給好雷的電影很簡單只要能讓人投入劇情內又有異想不到的驚喜這兩點就夠了投入基本就是運用你的資源你的演員流暢的說一個合理的故事更深入就用畫面或剪輯或用音樂歌曲讓觀眾沈浸在其中相信這需要製作方大量的經驗與深厚的功力才能成為上乘作品但是驚喜便是輕鬆額外加分的關鍵有勾人發噱的笑點有想破頭也想不到可以這樣演的原創橋段有翻轉公式的精心設計最讓我叫好鼓掌的是那些勇於挑戰的所謂badass橋段看電影可以覺得good可以覺得great覺得好到無以名狀我認為更高於前者的就是badassbadass就是你叫我不能這麼做我偏要做還要做的漂亮做的讓觀眾爽更甚者能做到killingthebusiness無人能出其右打破舊觀念是簡單的做法超人扭斷薩德的脖子挑戰觀影底線是常見做法萬磁王讓一顆硬幣穿腦而出打破第四道牆講一些五四三更是討喜的做法這些手法全都有爭議不可能獲得全部人的掌聲但是可看出製作方都有野心也都有膽子去實行我欣賞這股勇氣漫威製作的系列電影有一定的品質為了闔家觀賞的目標會更大眾化一點但他們仍然敢玩一些與眾不同的點子直接開誠佈公說我是鋼鐵人或是打boss的時候跳舞打boss用盧的或直接把湯瑪士小貨車的臉轉個九十度放大幾百倍我都很欣賞也很享受只可惜黑豹電影以上一個優點都沒有黑豹電影是一個玩爛公式用錯公式不敢挑戰的安全牌或許電影它背負的期待太大包袱之重造成很多地方都不敢拍也許它說哦國家以外的人很苦也許它說我們回家了你就可以盡情的發揮聯想但是電影本身就像櫥窗內的人體模特兒一樣放棄媒體天生的話語權影評們要它變黑豹就給它黑色要它變金豹就給他金色但終究是空談這個模特兒不會動不會表達自己的意思更誇張的是丟個假髮脫個高跟鞋這再普通不過的

處理 [  負雷] 隨便拍拍隨便買單的「黑豹」 /bbs/movie/M.1518934190.A.F7B.html
隨便拍拍隨便買單的黑豹
我已經不太懂我進場看的是漫威這塊牌子還是看電影現在的作法就是倒果為因個人片隨便拍一拍告訴你這個免洗超級英雄會在復仇者聯盟尬一角美隊的超高人氣是橫跨了多部電影的成本沒錯從美ㄧ復ㄧ美二逐漸架構這個被時代拋下的超級英雄但是黑豹作為個人專屬的第一集電影可以說是毫無章法的亂塞一通一個陌生的人物要讓觀眾認識並喜愛我們和他一起緣起成長蛻變一起面對超級英雄被賦予的重量就像我們看鋼鐵人美隊一樣索爾姑且算進來吧返校日可以略過大部分的起步階段很大的原因是我們對蜘蛛人已經太熟太熟了況且返校日在著墨青少年在面對自己巨大能力的體會配著輕鬆詼諧的劇情著實是款佳作而黑豹並沒那種本錢美三提早出場不代表可以隨意帶過英雄崛起的過程整部片的構成就是什麼都來一點未來科幻來一點原始肉搏來一點特務出勤來一點飛車追逐來一點再配上社會問題當調味最後加一點漫威式的笑點就可以端上桌了賣錢了嚐起來就是可怕的樣版英雄片說不好吃的人還會被說沒有仔細吃呢整部片可以挑缺點的地方太多太多了打戲太多人噴就甭提了文戲的議題當然沒有問題有問題的是片子拍的爛議題跟電影油水分離是要討論個毛線我這是在看電影要有引人入勝的效果你想表達的議題才有意義不然我去看社論就好看啥小電影議題的重要性不會提升電影的評價因為那是因果錯亂的瞎捧就像是你不會說塞在大便裡的金莎好吃一樣意思
處理 [負雷] 只有政治正確的  黑豹 /bbs/movie/M.1518809750.A.4E0.html
只有政治正確的黑豹
昨日晚場看完黑豹後我有點無言我一度懷疑我看了什麼時間不短劇情簡單但想不起有什內容甚至想不起有什麼記憶深刻的畫面至於精彩的部分我想大約780左右都在預告中出現實在不意外畫面的部分就幾個地點輪轉基地實驗室內城市上空挑戰的崖邊地鐵然後最後在野戰的地方結束我估計黑豹的場景約90都是棚拍而且動畫感很重感覺沒什意思看黑豹衣上身以為看到形魔女至於黑豹的能力除了擋子彈再來就是炸開這部分非常模仿神力女超人尤其當cia特務在飛機上雙手合抱使出炸開時根本就是神力女超人的動作看完覺得真的很虛然後整部片都是政治正確例如女人強過男人黑人強過白人第三世界國家有著強權國家沒有的黑科技說實在蠻失望的
處理 [負雷]黑豹：除了美術視覺，其餘毫無魅力

處理 [負雷] 真的過譽的黑豹......... /bbs/movie/M.1518536086.A.B1E.html
真的過譽的黑豹
如果片商敢打著漫威史上最佳那無疑是過譽了甚至我幾乎從沒給過超人英雄電影負雷而黑豹可能是第一部在我眼裡雷神也許無趣但不會讓人失望而黑豹作到了他可能是我現在心目中漫威最讓人失望的電影先說優點大家大概也看過別人這麼說了音樂不錯美術不錯但我覺得這部分加分有限因為阿斯嘉天堂島薩卡星我們都看過了服裝我覺得整體很棒後面有些特效還不賴演員算稱職ok沒了缺點呢一個娛樂大片的兩大致命傷都有打鬥超難看劇情超平凡簡陋大抵想去看黑豹的多半是因為美隊3跟冬兵美隊的近身肉搏看起來很俐落很帥但這部片等於通通沒有等級大概就黑暗騎士第三集華爾街的混混鬥毆天堂島的女兵們好看多了每一集的wwe套招都比他好看除了最後大亂鬥跟金銀豹喜大戰有用部份特效的部分外其他肉搏真的很糟套招還套的慢吞吞重點是還來回的很缺乏力道感至少諾蘭還說他想呈現的是真實有用力道上去的打鬥這次黑豹實在沒有我看johncena揮空拳都套的更有這麼一回事從一開始的山頭族王下來挑戰到後來韓國賭場亂鬥再到堂弟挑戰王位場場都讓我覺得很虛根本入不了戲覺得這是最強鬥士賭上生死的大戰啊啊一邊看一邊吐槽幹瓦干達是沒人了嗎這等級決鬥搶國王館長去不就得重新做一套xxlsize黑豹裝了嗎完全沒有透過電影技術剪接武指等讓大家信服他們夠強你是要告訴我黑豹的巔峰就是追捕冬兵那時候了嗎打鬥很糟劇情如果更簡陋整體問題就更不堪設想了因為黑豹講究肉搏他有很多劇情是靠決鬥推進決鬥如果很虛弱本來很簡陋的劇情就會顯得更像辦家家酒而就我而言黑豹劇情的簡陋在於每個角色都像玩rpg一樣時間到了出來講兩句台詞既不深刻也毫無情緒所以即使演員認真演角色深度就是很薄弱很沒有血肉感前面鋪陳的好像是堂弟的情人蠻重要的結果要殺克勞時被殺堂弟一點悲傷的劇情或畫面都沒有然後你最後在那裡說我最愛的人都死了這句話能帶給觀眾什麼重量嗎陣守邊界的大將姑且叫他pennyhardaway吧有87像一開始還一副以他跟黑豹的交情願意為他出生入死等到劇情需要只不過沒幫他一次把克勞抓回來就直接嗆前後兩任國王是怎樣然後劇情需要轉瞬間penny就反叛最後還殺的狠的比誰都兇隨便一個外人進來喔喔喔新國王就跟從心路歷程毫無鋪陳根本就變成為反而反堂弟也是一樣突然某天才想到可以去瓦干達他拿著他爸的遺物隨時

處理 [負雷] 四不像的黑豹 /bbs/movie/M.1527918611.A.56C.html
四不像的黑豹
首先從主角來看這個英雄就夠膚淺了失去了黑科技跟禁藥就是魯蛇甚至連奪回禁藥跟黑科技都不是透過自己努力是妹妹和前女友幫他準備好的終究就是一個少爺型的英雄在困境中沒有獲得任何成長不像諾蘭的蝙蝠俠原本是高高在上的少爺不斷地在挫折中成長和奮鬥也不像鋼鐵人和索爾即使失去最強的武器也能透過英雄的意志反敗為勝再來說金豹比種族主義者還腦殘在他的想法裡黑人有事歐巴馬就要救援黃人有事習近平就要救援白人有事梅克爾就要救援他把種族主義縮小成三種顏色的膚色主義更是無可救藥的愚蠢至於打鬥沒有異能的黑寡婦隨便一個體術都比黑豹帥氣幾百倍就爽片來講也完全不合格基本上就是拍給非裔美人看的爽片跟拍給共產政權下的華人爽片戰狼沒什麼不同能有超高票房的理由是完全一樣的
處理 [微負雷] 黑豹有點讓人失望.... /bbs/movie/M.1527839684.A.EF0.html
黑豹有點讓人失望
前陣子工作太忙完全沒空去看復仇者三最近案子告一段落有一小段休假想說先把前面幾部還沒看過的漫威追完再來看大亂鬥所以第一部就選了黑豹來看看完之後感受有點複雜個人的評價覺得黑豹在漫威電影裡面只有中下的程度大概有幾點想跟大家討論第一電影裡的角色設定感覺不太合理雖然主角是黑豹但在劇中反而金豹的人物性格比較鮮明黑豹本人就讓我覺得嗯沒甚麼國王的樣子第二黑豹跟其他漫威電影比起來好像少了讓人印象深刻的打鬥場面而且角色的能力好像有點弱除了貓咪服跟爪子抓抓以外沒甚麼特別的我就不說美國隊長了第三瓦干達明明就是一個充滿高科技的大國貓咪服超強結界神奇救命醫療裝置等等卻還是用非常傳統的方式拿一條項鍊就可以挑戰國王埋進東西立面才能變身的設定也有點讓人問號後面那個明明就是冰雖然好像很多討論是說這部電影強調的是一些政治議題或是人權種族等等裡面也有多關於革命或是正義的一些反思但還是覺得英雄片的前提是要夠帥夠強夠爽阿阿阿大家對這部片的評價如何呢其實不太懂為什麼它票房這麼好另外大家現在都在哪看這些比較新的電影阿以前習慣在dvd店租片現在連我家附近僅存的亞藝都倒掉了qq
處理 [負雷] 是不是有精神分裂的黑豹 /bbs/movie/M.1526734538.A.5E2.html
是不是有精神分裂的黑豹
整部故事主軸圍繞在擁有黑科技的國家該不該把科技

處理 [  負雷] 隨便拍拍隨便買單的「黑豹」 /bbs/movie/M.1518934190.A.F7B.html
隨便拍拍隨便買單的黑豹
我已經不太懂我進場看的是漫威這塊牌子還是看電影現在的作法就是倒果為因個人片隨便拍一拍告訴你這個免洗超級英雄會在復仇者聯盟尬一角美隊的超高人氣是橫跨了多部電影的成本沒錯從美ㄧ復ㄧ美二逐漸架構這個被時代拋下的超級英雄但是黑豹作為個人專屬的第一集電影可以說是毫無章法的亂塞一通一個陌生的人物要讓觀眾認識並喜愛我們和他一起緣起成長蛻變一起面對超級英雄被賦予的重量就像我們看鋼鐵人美隊一樣索爾姑且算進來吧返校日可以略過大部分的起步階段很大的原因是我們對蜘蛛人已經太熟太熟了況且返校日在著墨青少年在面對自己巨大能力的體會配著輕鬆詼諧的劇情著實是款佳作而黑豹並沒那種本錢美三提早出場不代表可以隨意帶過英雄崛起的過程整部片的構成就是什麼都來一點未來科幻來一點原始肉搏來一點特務出勤來一點飛車追逐來一點再配上社會問題當調味最後加一點漫威式的笑點就可以端上桌了賣錢了嚐起來就是可怕的樣版英雄片說不好吃的人還會被說沒有仔細吃呢整部片可以挑缺點的地方太多太多了打戲太多人噴就甭提了文戲的議題當然沒有問題有問題的是片子拍的爛議題跟電影油水分離是要討論個毛線我這是在看電影要有引人入勝的效果你想表達的議題才有意義不然我去看社論就好看啥小電影議題的重要性不會提升電影的評價因為那是因果錯亂的瞎捧就像是你不會說塞在大便裡的金莎好吃一樣意思
處理 [負雷] 只有政治正確的  黑豹 /bbs/movie/M.1518809750.A.4E0.html
只有政治正確的黑豹
昨日晚場看完黑豹後我有點無言我一度懷疑我看了什麼時間不短劇情簡單但想不起有什內容甚至想不起有什麼記憶深刻的畫面至於精彩的部分我想大約780左右都在預告中出現實在不意外畫面的部分就幾個地點輪轉基地實驗室內城市上空挑戰的崖邊地鐵然後最後在野戰的地方結束我估計黑豹的場景約90都是棚拍而且動畫感很重感覺沒什意思看黑豹衣上身以為看到形魔女至於黑豹的能力除了擋子彈再來就是炸開這部分非常模仿神力女超人尤其當cia特務在飛機上雙手合抱使出炸開時根本就是神力女超人的動作看完覺得真的很虛然後整部片都是政治正確例如女人強過男人黑人強過白人第三世界國家有著強權國家沒有的黑科技說實在蠻失望的
處理 [負雷]黑豹：除了美術視覺，其餘毫無魅力

處理 [負雷] 真的過譽的黑豹......... /bbs/movie/M.1518536086.A.B1E.html
真的過譽的黑豹
如果片商敢打著漫威史上最佳那無疑是過譽了甚至我幾乎從沒給過超人英雄電影負雷而黑豹可能是第一部在我眼裡雷神也許無趣但不會讓人失望而黑豹作到了他可能是我現在心目中漫威最讓人失望的電影先說優點大家大概也看過別人這麼說了音樂不錯美術不錯但我覺得這部分加分有限因為阿斯嘉天堂島薩卡星我們都看過了服裝我覺得整體很棒後面有些特效還不賴演員算稱職ok沒了缺點呢一個娛樂大片的兩大致命傷都有打鬥超難看劇情超平凡簡陋大抵想去看黑豹的多半是因為美隊3跟冬兵美隊的近身肉搏看起來很俐落很帥但這部片等於通通沒有等級大概就黑暗騎士第三集華爾街的混混鬥毆天堂島的女兵們好看多了每一集的wwe套招都比他好看除了最後大亂鬥跟金銀豹喜大戰有用部份特效的部分外其他肉搏真的很糟套招還套的慢吞吞重點是還來回的很缺乏力道感至少諾蘭還說他想呈現的是真實有用力道上去的打鬥這次黑豹實在沒有我看johncena揮空拳都套的更有這麼一回事從一開始的山頭族王下來挑戰到後來韓國賭場亂鬥再到堂弟挑戰王位場場都讓我覺得很虛根本入不了戲覺得這是最強鬥士賭上生死的大戰啊啊一邊看一邊吐槽幹瓦干達是沒人了嗎這等級決鬥搶國王館長去不就得重新做一套xxlsize黑豹裝了嗎完全沒有透過電影技術剪接武指等讓大家信服他們夠強你是要告訴我黑豹的巔峰就是追捕冬兵那時候了嗎打鬥很糟劇情如果更簡陋整體問題就更不堪設想了因為黑豹講究肉搏他有很多劇情是靠決鬥推進決鬥如果很虛弱本來很簡陋的劇情就會顯得更像辦家家酒而就我而言黑豹劇情的簡陋在於每個角色都像玩rpg一樣時間到了出來講兩句台詞既不深刻也毫無情緒所以即使演員認真演角色深度就是很薄弱很沒有血肉感前面鋪陳的好像是堂弟的情人蠻重要的結果要殺克勞時被殺堂弟一點悲傷的劇情或畫面都沒有然後你最後在那裡說我最愛的人都死了這句話能帶給觀眾什麼重量嗎陣守邊界的大將姑且叫他pennyhardaway吧有87像一開始還一副以他跟黑豹的交情願意為他出生入死等到劇情需要只不過沒幫他一次把克勞抓回來就直接嗆前後兩任國王是怎樣然後劇情需要轉瞬間penny就反叛最後還殺的狠的比誰都兇隨便一個外人進來喔喔喔新國王就跟從心路歷程毫無鋪陳根本就變成為反而反堂弟也是一樣突然某天才想到可以去瓦干達他拿著他爸的遺物隨時

In [108]:
id_to_body = {**d1,**d2}

In [112]:
with open('id_to_body.json','w',encoding='utf-8')as f:
    json.dump(id_to_body, f, indent=2,ensure_ascii=False)

In [116]:
import jieba

def load_data(a_csv, b_json, label):
    a_ids=[]
    with open(a_csv, 'r', encoding='utf-8')as f:
        reader = csv.DictReader(f)
        for row in reader:
            a_ids.append(row['href'])
    with open(b_json, 'r', encoding='utf-8')as f:
        id_to_body = json.load(f)
    data = []
    for a_id in a_ids:
        tokenized_post = []
        txt = id_to_body[a_id]
        for sent in txt.split():
            filtered = [t for t in jieba.cut(sent) if t.split() and len(t)>1]
            tokenized_post += filtered
        data.append((tokenized_post,label))
    return data

In [117]:
pos_data = load_data('mov_pos.csv', 'id_to_body.json','正評')

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/xm/lf91wnm15nx232z9w7y41f5w0000gn/T/jieba.cache
Loading model cost 0.852 seconds.
Prefix dict has been built successfully.


In [134]:
for post, label in pos_data[:3]:
    print(post[:5],label)

['比上', '沒得', '比比', '下有', '黑豹'] 正評
['暴走', '曼哈頓', '黑豹', '化身', '正義猛警'] 正評
['黑豹', '非洲', '先進', '文明', '卓然'] 正評


In [121]:
neg_data = load_data('mov_neg.csv','id_to_body.json','負評')
len(neg_data)

56

In [170]:
import random
random.seed(42)
random.shuffle(pos_data)
random.shuffle(neg_data)
x_train, y_train, x_test, y_test = [],[],[],[]
for i in range(45):
    x_train.append(''.join(pos_data[i][0]))
    x_train.append(''.join(neg_data[i][0]))
    y_train.append(pos_data[i][1])
    y_train.append(neg_data[i][1])
    
for i in range(45,len(neg_data)):
    x_test.append(''.join(pos_data[i][0]))
    x_test.append(''.join(neg_data[i][0]))
    y_test.append(pos_data[i][1])
    y_test.append(neg_data[i][1])
    
    
print(y_train[0])

正評


In [168]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

In [163]:
vectorizer = CountVectorizer()
x_train = vectorizer.fit_transform(x_train)
transformer = TfidfTransformer()
x_train = transformer.fit_transform(x_train)
clf = SGDClassifier(random_state=42)
clf.fit(x_train,y_train)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [164]:
x_test = vectorizer.transform(x_test)
x_test = transformer.transform(x_test)
y_pred = clf.predict(x_test)
print('predict:',list(y_pred))
print('ground_truth:',y_test)
print(accuracy_score(y_test,y_pred))

predict: ['正評', '負評', '正評', '正評', '正評', '負評', '正評', '負評', '正評', '負評', '正評', '正評', '正評', '負評', '正評', '負評', '正評', '正評', '正評', '負評', '正評', '正評']
ground_truth: ['正評', '負評', '正評', '負評', '正評', '負評', '正評', '負評', '正評', '負評', '正評', '負評', '正評', '負評', '正評', '負評', '正評', '負評', '正評', '負評', '正評', '負評']
0.8181818181818182


In [171]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(x_train)
transformer = TfidfTransformer()
X_train = transformer.fit_transform(X_train)
X_test = vectorizer.transform(x_test)
X_test = transformer.transform(X_test)

In [175]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
# Specify L1 regularization
lr = LogisticRegression()

# Instantiate the GridSearchCV object and run the search
searcher = GridSearchCV(lr, {'C':[0.001, 0.01, 0.1, 1, 10]})
searcher.fit(X_train, y_train)
y_pred = searcher.predict(X_test)

# Report the best parameters
print("Best CV params", searcher.best_params_)

# Find the number of nonzero coefficients (selected features)
best_lr = searcher.best_estimator_
coefs = best_lr.coef_
print("Total number of features:", coefs.size)
print("Number of selected features:", np.count_nonzero(coefs))
print("accuracy of logistic regression:",searcher.score(X_test,y_test))
print(classification_report(y_test, y_pred))
print(coefs.view)

Best CV params {'C': 0.001}
Total number of features: 64
Number of selected features: 64
accuracy of logistic regression: 0.8181818181818182
              precision    recall  f1-score   support

          正評       0.73      1.00      0.85        11
          負評       1.00      0.64      0.78        11

    accuracy                           0.82        22
   macro avg       0.87      0.82      0.81        22
weighted avg       0.87      0.82      0.81        22

<built-in method view of numpy.ndarray object at 0x128597580>


In [178]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
lsvc = LinearSVC(C=0.1, penalty="l1", dual=False)
lsvc2 = lsvc.fit(X_train, y_train)
model = SelectFromModel(lsvc2, prefit=True)
X_new_svm = model.transform(X_train)
y_pred_svc = lsvc2.predict(X_test)
print(X_new_svm.shape)
print('Number of selected features:',X_new_svm.shape[1])
print('accuracy of svm:',lsvc2.score(X_test, y_test))
print(classification_report(y_test, y_pred_svc))

(90, 0)
Number of selected features: 0
accuracy of svm: 0.5
              precision    recall  f1-score   support

          正評       0.50      1.00      0.67        11
          負評       0.00      0.00      0.00        11

    accuracy                           0.50        22
   macro avg       0.25      0.50      0.33        22
weighted avg       0.25      0.50      0.33        22

